<a href="https://colab.research.google.com/github/omidshy/ML/blob/master/src/MB-potential.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Modeling the Müller-Brown PES using a Neural Network

In this tutorial, we will learn how to use a neural network (NN) model to predict the energy of points on the Müller-Brown potential energy surface.


In [ ]:
!pip install numpy
!pip install matplotlib
!pip install plotly
!pip install torch

In [ ]:
from math import exp, pow, tanh
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

## Defining the Müller-Brown Potential

Bellow you can see the functional form of the Müller-Brown potential. For more details see [here](https://www.wolframcloud.com/objects/demonstrations/TrajectoriesOnTheMullerBrownPotentialEnergySurface-source.nb).

$v(x,y) = \sum_{k=0}^3 A_k \mathrm{exp}\left[ a_k (x - x_k^0)^2 + b_k (x - x_k^0) (y - y_k^0) + c_k (y - y_k^0)^2 \right] $

First, we define the Müller-Brown potential as a function:

In [ ]:
def mueller_brown_potential(x, y):
  A = [-200, -100, -170, 15]
  a = [-1, -1, -6.5, 0.7]
  b = [0, 0, 11, 0.6]
  c = [-10, -10, -6.5, 0.7]
  x0 = [1, 0, -0.5, -1.0]
  y0 = [0, 0.5, 1.5, 1]
  value = 0
  for k in range(0, 4):
    # Scale the function by 0.1 to make plotting easier.
    value += 0.1 * A[k] * exp( a[k] * pow(x-x0[k], 2.0) + b[k] * (x-x0[k]) * (y-y0[k]) + c[k] * pow(y-y0[k], 2.0))
  return value

## Generate Training Data

Next, we need to generate data points to train the neural network. The training data will be generated using the Müller-Brown Potential and a range of x and y values.

In [ ]:
# Generate a set of x and y values on a regular grid
xx = np.arange(-1.8, 1.4, 0.1)
yy = np.arange(-0.4, 2.4, 0.1)
X, Y = np.meshgrid(xx, yy)

xy, xy_truncated = [], []
e, e_truncated = [], []

for y in yy:
  for x in xx:
    v = mueller_brown_potential(x,y) # Now using x and y values from the xx and yy arrays.
    e.append(v) # Storing potential energy values in the 'e' array.
    xy.append([x,y])
    if v < 10:  # Keep only low-energy points for training.
      xy_truncated.append([x,y])
      e_truncated.append(v)

# Reshape 'e' array so that we can plot our data on a 2-D surface that is len(xx) by len(yy).
E = np.reshape(e,(len(yy),-1))

print("E_min:", np.amin(E), "E_max:", np.amax(E))
print("Size of test set:", len(e))
print("Size of training set:", len(e_truncated))

### Visualizing Data: 3D Surface

We will now create a 3-D plot of our training data. To make the plot more readable, we will replace the points that have extremely high energy with nan (not a number). This will keep our $Z$ array the same shape and help us ignore the high energy region that we are not interested in.

In [ ]:
fig = go.Figure(data=[go.Surface(z=E, x=X, y=Y, colorscale='rainbow', cmin=-15, cmax=9)])
fig.update_traces(contours_z=dict(show=True, project_z=True))
fig.update_layout(title='Mueller-Brown Potential', width=500, height=500,
                  scene = dict(
                  zaxis_title="E",
                  zaxis = dict(dtick=3, range=[-15, 15]),
                  camera_eye = dict(x=-1.2, y=-1.2, z=1.2)
                  ))
fig.show()

### Visualizing Data: Contour Surface

To allow for an easier visualization of the potential energy surface, we can generate a 2-D contour surface.

In [ ]:
fig = plt.figure(figsize=(3,2.5), dpi=150)
levels = [-12, -8, -4, 0, 4, 8, 10]
ct = plt.contour(X, Y, E, levels, colors='k')
plt.clabel(ct, inline=True, fmt='%3.0f', fontsize=8)
ct = plt.contourf(X, Y, E, levels, cmap=plt.cm.rainbow, extend='both', vmin=-15, vmax=0)
plt.xlabel("x", labelpad=-0.75)
plt.ylabel("y", labelpad=2.5)
plt.tick_params(axis='both', pad=2,labelsize=8)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=8)
plt.title('Müeller Brown Contour Surface', fontsize=8)
plt.tight_layout()
plt.show()

## Loading PyTorch and Training Data

After installing and importing pytorch, we will save our training data as a tensor data set.


In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
from torch.utils.data import TensorDataset, DataLoader, random_split

dataset = TensorDataset(Tensor(xy_truncated), Tensor(e_truncated))
train_dataset, test_dataset = random_split(dataset, [0.7, 0.3])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
print("Size of training set:", len(train_loader.dataset))

### Defining the Neural Network Class

Here we define our neural network as a Python class. A function ($\it{train\_loop}$) is used to loop through our training data.

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, n=20):  #n is the number of neurons for the first layer
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(2,n), # Linear function taking 2 inputs and outputs data for n neurons.
            nn.Tanh(),
            nn.Linear(n, 1) # Linear function taking data from n neurons and producing one output value.
        )

    def forward(self, x):
        return self.model(x)

def train_loop(dataloader, model, loss_fn, optimizer, epoch):

    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred.squeeze(), y)

        # Backpropagation - using the gradients of the loss function to update the weights and biases.
        optimizer.zero_grad() # Zero out the gradients from the previous iteration to replace them.
        loss.backward() # Update the gradients of the loss function.
        optimizer.step() # Uses step() to optimize the weights and biases with the updated gradients.

        if batch % 32 == 0 and epoch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"epoch: {epoch:>4d} loss: {loss:>7.3f}  [{current:>5d}/{size:>5d}]")

## Training the Model

Now we can train the neural network model. We will finish our training when the desired number of epochs has been reached. We will also define other hyper-parameters used for the training.

In [ ]:
n_hidden = 32
learning_rate = 1e-2
epochs = 2000

model = NeuralNetwork(n_hidden)
loss_fn = F.mse_loss
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    train_loop(train_loader, model, loss_fn, optimizer, epoch)

print("Done with Training!")

## Plotting Reference, Predicted, and Difference Surfaces

Finally, we will plot the Müller-Brown potential energy surface using the analytical function (reference), using the predicted values from the neural network (predicted), and we will show the difference between the predicted and reference surfaces.

In [ ]:
def show_surface(model):

  e_pred = model(Tensor(xy))
  E_pred = np.reshape(e_pred.detach().numpy(),(len(yy),-1))
  Ediff = np.subtract(E_pred, E)

  fig = plt.figure(figsize=(3,7.5), dpi=150)


  plt.subplot(3, 1, 1)
  levels = [-12, -8, -4, 0, 4, 8]
  ct = plt.contour(X, Y, E, levels, colors='k')
  plt.clabel(ct, inline=True, fmt='%3.0f', fontsize=8)
  ct = plt.contourf(X, Y, E, levels, cmap=plt.cm.rainbow, extend='both', vmin=-15, vmax=0)
  plt.title("(a) Reference", fontsize=8)
  plt.xlabel("x", labelpad=-0.75)
  plt.ylabel("y", labelpad=2.5)
  plt.tick_params(axis='both', pad=2,labelsize=8)
  cbar= plt.colorbar()
  cbar.ax.tick_params(labelsize=8)

  plt.subplot(3, 1, 2)
  ct = plt.contour(X, Y, E_pred, levels, colors='k')
  plt.clabel(ct, inline=True, fmt='%3.0f', fontsize=8)
  ct = plt.contourf(X, Y, E_pred, levels, cmap=plt.cm.rainbow, extend='both', vmin=-15, vmax=0)
  plt.title("(b) Predicted", fontsize=8)
  plt.xlabel("x", labelpad=-0.75)
  plt.ylabel("y", labelpad=2.5)
  plt.tick_params(axis='both', pad=2,labelsize=8)
  cbar= plt.colorbar()
  cbar.ax.tick_params(labelsize=8)

  plt.subplot(3, 1, 3)
  levels = [-4, -2, 0, 2, 4]
  ct = plt.contour(X, Y, Ediff, levels, colors='k')
  plt.clabel(ct, inline=True, fmt='%3.0f', fontsize=8)
  ct = plt.contourf(X, Y, Ediff, levels, cmap=plt.cm.rainbow, extend='both', vmin=-4, vmax=4)
  plt.title("(c) Difference", fontsize=8)
  plt.xlabel("x", labelpad=-0.75)
  plt.ylabel("y", labelpad=2.5)
  print("diff, min, max:", np.amin(Ediff), np.amax(Ediff))
  plt.tick_params(axis='both', pad=2,labelsize=8)
  cbar= plt.colorbar()
  cbar.ax.tick_params(labelsize=8)

  plt.tight_layout()

  plt.show()

show_surface(model)